In [25]:
import pandas as pd
import numpy as np
from scipy.stats import poisson

selecoes = pd.read_excel(
    "DadosCopaDoMundoQatar2022.xlsx", sheet_name="selecoes", index_col=0
)

jogos = pd.read_excel("DadosCopaDoMundoQatar2022.xlsx", sheet_name="jogos")


In [14]:
jogos.head()

In [26]:
# carreganndo os dados de pontuação da FIFA
fifa = selecoes["PontosRankingFIFA"]

# criando uma transformação lienar para os valores de escala numérica

a, b = min(fifa), max(fifa)
fa, fb = 0.15, 1
# fazendo uma transformação linear, com 0.15 o menor valor e 1 o maior
# calcula o coef. de inclinação da reta e b0
b1 = (fb - fa) / (b - a)
b0 = fb - b*b1

forca = b1 * fifa + b0
# forca = ranking FIFA recalculado

In [33]:
def MediaPoisson(selecao1, selecao2) -> list[int, int]:
    forca1 = forca[selecao1]
    forca2 = forca[selecao2]

    # media de gols estimada para toda a Copa baseada nas últimas médias em copas anteriores
    mediaGolsEstimada = 2.75
    lambda1 = mediaGolsEstimada * forca1 / (forca1 + forca2)
    lambda2 = mediaGolsEstimada - lambda1
    
    return [lambda1, lambda2]

def Resultado(gols1, gols2):
    if gols1 > gols2:
        return "V"
    elif gols2 > gols1:
        return "D"
    return "E"


def Pontos(gols1: int, gols2: int) -> list:
    resultado = Resultado(gols1, gols2)
    
    if resultado == "V":
        pontos1, pontos2 = 3, 0
    elif resultado == "D":
        pontos1, pontos2 = 0, 3
    else:
        pontos1, pontos2 = 1, 1
        
    return [pontos1, pontos2, resultado]



In [154]:
def Jogo(selecao1: str, selecao2: str) -> list:
    # usando os dados do ranking FIFA

    # lambda = média
    lambda1, lambda2 = MediaPoisson(selecao1, selecao2)
    # simulando um jogo
    gols1 = int(np.random.poisson(lam=lambda1, size=1))
    gols2 = int(np.random.poisson(lam=lambda2, size=1))

    saldo1 = gols1 - gols2
    saldo2 = -saldo1

    pontos1, pontos2, resultado = Pontos(gols1, gols2)

    placar = f"{gols1}x{gols2}"

    return [gols1, gols2, saldo1, saldo2, pontos1, pontos2, resultado, placar]


# calcula probabilidade de 0-6 gols e 7+
# dá a média de gols e calcula a prbabilidade de cada resultado
def Distribuicao(media: int):
    probs = []
    # placar até 7 gols no máximo
    for i in range(7):
        probs.append(poisson.pmf(i, media))
    # probabilidade para 7 ou mais
    probs.append(1 - sum(probs))

    return pd.Series(probs, index=["0", "1", "2", "3", "4", "5", "6", "7+"])


Distribuicao(2)


def ProbabilidadesPartida(selecao1, selecao2) -> list:
    media1, media2 = MediaPoisson(selecao1, selecao2)
    dist1, dist2 = Distribuicao(media1), Distribuicao(media2)
    matriz = np.outer(dist1, dist2)

    # probabilidades
    vitoria = np.tril(matriz).sum() - np.trace(matriz)  # triangular superior - traço
    derrota = np.triu(matriz).sum() - np.trace(matriz)  #  tringular inferior - traço
    empate = 1 - (vitoria + derrota)

    probs = np.around([vitoria, empate, derrota], 3)
    probsp = [f"{100*i:.1f}%" for i in probs]  # %

    nomes = ["0", "1", "2", "3", "4", "5", "6", "7+"]
    matriz = pd.DataFrame(matriz, columns=nomes, index=nomes)
    matriz.index = pd.MultiIndex.from_product([[selecao1], matriz.index])
    matriz.columns = pd.MultiIndex.from_product([[selecao2], matriz.columns])

    output = {
        "selecao1": selecao1,
        "selecao2": selecao2,
        "força1": forca[selecao1],
        "força2": forca[selecao2],
        "media1": media1,
        "media2": media2,
        "probabilidades": probsp,
        "matriz": matriz,
    }

    return output


In [161]:
jogos["Vitória"] = None
jogos["Empate"] = None
jogos["Derrota"] = None

for i in range(jogos.shape[0]): # tamanho do conjunto de dados
    selecao1 = jogos["seleção1"][i]
    selecao2 = jogos["seleção2"][i]
    v, e, d = ProbabilidadesPartida(selecao1, selecao2)["probabilidades"]
    
    jogos.at[i, "Vitória"] = v
    jogos.at[i, "Empate"] = e
    jogos.at[i, "Derrota"] = d
    
jogos.to_excel("outputEstimativaJogosCopa.xlsx")
jogos
     
jogos

,data,hora,grupo,seleção1,seleção2,estádio,Vitória,Empate,Derrota
0,2022-11-21,07:00:00,A,Senegal,Holanda,AL THUMAMA,28.3%,24.9%,46.8%
1,2022-11-21,13:00:00,A,Catar,Equador,AL KHOR,32.3%,25.4%,42.3%
2,2022-11-25,10:00:00,A,Catar,Senegal,AL THUMAMA,17.2%,22.1%,60.6%
3,2022-11-25,13:00:00,A,Holanda,Equador,INTER. KHALIFA,64.4%,21.0%,14.6%
4,2022-11-29,12:00:00,A,Holanda,Catar,AL KHOR,68.4%,19.6%,12.0%
5,2022-11-29,12:00:00,A,Equador,Senegal,INTER. KHALIFA,20.7%,23.3%,56.0%
6,2022-11-21,10:00:00,B,Inglaterra,Irã,INTER. KHALIFA,54.6%,23.6%,21.8%
7,2022-11-21,16:00:00,B,Estados Unidos,País de Gales,AL RAYYAN,43.0%,25.3%,31.7%
8,2022-11-25,07:00:00,B,País de Gales,Irã,AL RAYYAN,40.1%,25.5%,34.4%
9,2022-11-25,16:00:00,B,Inglaterra,Estados Unidos,AL KHOR,46.1%,25.0%,28.9%
